## <span style="color:grey">Initialization</span>

In [6]:
import pandas as pd
import numpy as np
import warnings
import time
import datetime
import logging
import copy 
import os
from multiprocessing import Process
warnings.filterwarnings('ignore')

exchange = 'NSE'
symbol = "HDFC"

today = datetime.datetime.now()
#configuration for downloading data
days =  1
years = 5

if today.weekday() >= 5:
    today = today - datetime.timedelta(days = today.weekday() - 4)
    
#today = datetime.datetime(2019, 4, 8, 15, 45, 0)


toDate = today
#toDate = today - datetime.timedelta(days=3)
fromDate = toDate - datetime.timedelta(days = days)


#if today.weekday() == 0:
#    fromDate = toDate - datetime.timedelta(days = 3)

freq = '1min'

mode = "Algo" # Backtesting/Live/Algo
offline = False
if mode =="Live":
    offline = False

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"

from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0

%run "KiteConnect_Library.ipynb"    

### Kite - OAuth Login

In [2]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

logging.basicConfig(filename="log/live_log.log", filemode="a", level=logging.DEBUG, format="[%(asctime)s: %(levelname)s]:%(message)s")
logger=logging.getLogger()

kite = KiteConnect(api_key=KiteAPIKey)
reauthentication = False

f = open("kite_data/access_token.txt", mode="r")
access_token = f.readlines()
logger.info(access_token[0])
kite.set_access_token(access_token[0])

try:
    logger.info("Welcome "+kite.profile()['user_name'])
except:
    if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")
    driver.get(kite.login_url())
    reauthentication = True
    

In [3]:
if reauthentication:
    request_token = re.findall("request_token=(.*)&action",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    access_token = data["access_token"]
    kite.set_access_token(access_token)
    f = open("kite_data/access_token.txt", mode="w")
    f.write(access_token)
    f.close()
    driver.close()

### Download NSE Stock Master List

In [4]:
if offline != True:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange, mode='w')

instruments_df = pd.read_hdf('kite_data/kite_data.h5', key=exchange, mode='r')

EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

### Filter Stocks - portfolio maker

In [5]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

downloadlist = nifty50['Symbol']
industry = niftynext50['Industry'].unique()

##  <span style="color:green">Trading Strategies </span>

In [169]:
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0
iMACD = 0
iMACDhist = 0
iMACDsignal = 0
UP = 0
DOWN = 1
SD = 0
iRSI = 0 
atr = 0
STATE = "SELL"
sma20 = 0
sma50 = 0

# Methods
REF = lambda df, i: df.shift(i)
#TREND_UP = lambda a, days: ROC(a,days) >= 0.01
#TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01

TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)))
TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2)))
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
profit = 0
stoploss = 0
squareoff = 0

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        global iMACD, iMACDhist, iMACDsignal, SD, iRSI, sma20, sma50
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        AroonDown, AroonUp = AROON(HIGH, LOW, 25)
        iMACD, iMACDsignal, iMACDhist = MACDEXT(CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        SD = STDDEV(CLOSE)
        iRSI = RSI(CLOSE, timeperiod=14)
        sma20 = SMA(CLOSE, timeperiod=20)
        sma50 = SMA(CLOSE, timeperiod=50)
        #atr = ATR(CLOSE)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
     
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    def short_hanging_man(self):
        return pd.DataFrame(TREND_UP(CLOSE, 10) &(CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100), columns=["sell"])
    
    def short_bear_kicking(self):
        return pd.DataFrame(CDLKICKING(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
        
    def short_shooting_star(self):
        return pd.DataFrame( TREND_UP(CLOSE, 5) & CDLSHOOTINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
    
    def long_pa(self):
        return pd.DataFrame(TREND_DOWN(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == 100)
                            #| (CDLABANDONEDBABY(OPEN, HIGH, LOW, CLOSE) != 0) 
                            ), columns=["buy"])
    def short_pa(self):
        return pd.DataFrame( TREND_UP(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == -100)
                            #| (CDLEVENINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0)
                            ) , columns=["sell"])
    
    def tradeDecision(self):
        
        buy  = self.long_pa() # |self.long_macd_crossover() # | self.long_bb() #| self.long_ichimoku() #| self.long_bull_engulf_ema() #| self.long_ichimoku()
        
        sell = self.short_pa() # |self.short_macd_crossover() # | self.short_bb() #| self.short_bear_engulf() #| self.short_hanging_man() #|  self.short_ichimoku()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE
        buy['sl'] = CLOSE - SD
        sell['sl'] = CLOSE + SD
        buy['so'] = CLOSE + 4 * SD
        sell['so'] = CLOSE - 4 * SD

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
    
    
    def tradeBot(self, symbol):
        global STATE
        buy, sell = self.tradeDecision()
        logger.debug(symbol+","+STATE)
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        
        price = CLOSE[-1]
        trigger = CLOSE[-1]
        stoploss = SD[-1]
        squareoff = 3 * SD[-1] 
        
        logger.debug(buy)
        logger.debug(sell)
        
        if buy.empty | sell.empty:
            return
        
        if (buy.empty != True) & ((delT1 == buy.index[-1]) | (delT2 == buy.index[-1])) & (STATE != "SELL"):
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_slm(symbol, price, stoploss, quantity=1)
            STATE = "SELL"
        elif (sell.empty != True) & ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) & (STATE != "BUY"):
            STATE = "BUY"
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_slm(symbol, price, stoploss, quantity=1)
            
    def papertradeBot(self, symbol, timestamp):
        global STATE, profit, squareoff, stoploss
        buy, sell = self.tradeDecision()
        #logger.debug(buy)

        timeNow = timestamp
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)

        #logger.debug(timeNow)

        price = CLOSE[-1]
        trigger = CLOSE[-1]
        stoploss = price * (1-0.01)
        squareoff = 3 * SD[-1] 
        
        if (LOW[-1]<=stoploss) and (STATE =="SELL"):
            profit = profit - stoploss
            STATE = "BUY"
            return
        
        if (HIGH[-1]<=stoploss) and (STATE =="BUY"):
            profit = profit + stoploss
            STATE = "SELL"
            return
        #
        #if HIGH[-1] >= squareoff:
        #    profit = profit - squareoff
        #    STATE = "BUY"
        #    return

        if buy.empty | sell.empty:
            return
        
        if ((delT1 == buy.index[-1]) or (delT2 == buy.index[-1])) and (STATE != "SELL"):
            if STATE == "":
                profit = profit - price
            else:
                profit = profit - 2*price
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_slm(symbol, price, stoploss, quantity=1)
            STATE = "SELL"
        elif ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) and (STATE != "BUY"):
            if STATE == "":
                profit = profit + price
            else:
                profit = profit + 2*price
            STATE = "BUY"
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_slm(symbol, price, stoploss, quantity=1)
            
        
papertrade = False
if (mode=="Algo") & (papertrade == False):
    global fig
    noofcandles = 60
    x = "WIPRO"
    temp_data = downloadData(x, fromDate = toDate - dt.timedelta(days=2), toDate=toDate, freq="minute")
    
    #logger.debug(temp_data)
    #temp_data = downloadData(x, fromDate, toDate, "day")
    #logging.debug(temp_data)
        
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig = createPlot(x)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    #temp_data = aroon(temp_data,5, True)
    temp_data = stoch(temp_data,5, True)
    #temp_data = sma(temp_data,1,True)
    obv(temp_data,2, True)
    
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()

    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, True)


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



In [172]:
portfolio = pd.DataFrame(downloadlist)
portfolio = pd.DataFrame(['INFRATEL'])
papertrade = True
if papertrade:
    for index,symbol in portfolio.iterrows():
        #print(symbol[0])
        
        profit = 0
        STATE = "BUY"
        squareoff = 0
        stoploss = 0
        global fig
        noofcandles = 60
        x = symbol[0]
        temp_data = downloadData(x, fromDate = toDate - dt.timedelta(days=2), toDate=toDate, freq="minute")
        rowSize = temp_data.shape[0]

        for i in np.linspace(120,rowSize-1, rowSize-121, dtype="int64"):
            a = algoTrade(temp_data.iloc[(i-120):i])
            a.papertradeBot(x, temp_data.index[i])
            #print(temp_data.iloc[(i-30):i])
        if STATE == "SELL":
            profit = profit + 2*temp_data.iloc[-1].close

        print(x+": "+str(round(profit,2))+", ("+str(round(profit/temp_data.iloc[-1].close*100,4))+" %)")

INFRATEL: -5.1, (-1.6307 %)


In [119]:
temp_data.iloc[-1].close

3074.9

## <span style="color:blue">BackTesting</span>

In [17]:
logger.setLevel(logging.DEBUG)

### Multithreaded

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
from multiprocessing import Pool
def threadedBatchAnalysis(x):
    fig = ""
    temp_data = downloadData(x, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
    #logging.debug(temp_data)
        
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig = createPlot(x)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    #print(temp_data.head())
    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    temp_data = aroon(temp_data,5, True)
    obv(temp_data,2, True)

    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    #print(os.getcwd())
    plotData(x, True)

if __name__ == '__main__':
    #if True | (mode == "Algo"):
    logger.debug("Hello")
    p = Pool(10)
    p.map_async(threadedBatchAnalysis, portfolio[0])
    #p.apply_async(threadedBatchAnalysis, ['SBIN'])
    #portfolio
    
#threadedBatchAnalysis()

In [33]:
import gc
gc.get_count()

(64, 1, 6)

In [34]:
gc.collect()

15062

### Backtesting - Single Threaded

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
def batchAnalysis(portfolio=portfolio):
    global fig, noofcandles
    
    for index, row in portfolio.iterrows():
        fig = ""
        symbol = row[0] #row.iloc[:,0]
        temp_data = downloadData(symbol, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
        
        #temp_data['symbol'] = symbol
        #temp_data.set_index(['temp_data.index], inplace=True)
        #logging.debug(temp_data)
        
        yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
        yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()
        
        xMin = temp_data.index[-1*noofcandles]
        xMax = temp_data.index[-1]
        fig = createPlot(symbol)
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['yaxis']['range'] = [xMin, xMax]

        #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
        temp_data = candlestick(temp_data,1,True)
        temp_data = bbands(temp_data,1, True)

        temp_data = macd(temp_data,3,True)
        temp_data = rsi(temp_data,4, True)
        temp_data = aroon(temp_data,5, True)
        obv(temp_data,2, True)
        
        a = algoTrade(temp_data)
        buy, sell = a.tradeDecision()
        buy_df = annotateBuySell(buy, "Buy")
        sell_df = annotateBuySell(sell, "Sell")
        
        plotData(symbol)
        #input("Press Enter to continue")
    

batchAnalysis()

NetworkException: Request preparation failed (kitetrade-common).

### Order Management

In [54]:
papertrade =True

#logging.critical("BUY"+symbol)
def buy_slm(symbol, price, trigger,quantity=1): 
    logging.critical("BUY: "+symbol+", price: "+str(price)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    
        
def buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    logging.critical("BUY: "+symbol+", price: "+str(price)+", squareoff: "+str(squareoff)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))


def sell_slm(symbol, price, trigger, quantity=1):
    
    logging.critical("SELL: "+symbol+", price: "+str(price)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))

def sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    logging.critical("SELL: "+symbol+", price: "+str(price)+", squareoff: "+str(squareoff)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    
